# Benchmark

**!** This notebook has to be run after the following notebooks:
- [dataset.ipynb](dataset.ipynb)
- [scoring.ipynb](scoring.ipynb)
- [model.ipynb](model.ipynb)

In [1]:
import model
import dataset
import scoring

In [2]:
models = model.load_models()

| using constant padding
| using scales: [0.8333333333333334, 1.0, 1.2]
| using ordinary correlation
load pretrained model from models/SiamSE/checkpoint_vot.pth
remove prefix "module."
missing keys:set()
unused checkpoint keys:set()


In [3]:
datasets = dataset.load_datasets()

Error while loading dataset David gt and image files have different length
Error while loading dataset Football1 gt and image files have different length
Error while loading dataset Jogging1 gt and image files have different length
Error while loading dataset Jogging2 gt and image files have different length
Error while loading dataset Subway gt and image files have different length


In [4]:
models.keys(), datasets.keys()

(dict_keys(['SEsiamFC']), dict_keys(['mytc128', 'myvot2021']))

# Tracking

In [8]:
t1 = models['SEsiamFC']
d1 = datasets['mytc128']
v1 = d1['Cup']

In [9]:
import cv2
import numpy as np
import json
import os

In [24]:
def track_video(tracker, video, verbose=0, save_video=None):
    start_frame, toc = 0, 0

    pred = []
    image_files, gt = video['image_files'], video['gt']

    if save_video is not None:
        sh = cv2.imread(image_files[0]).shape[:2]
        frame_size = (sh[1], sh[0])
        out = cv2.VideoWriter(save_video, cv2.VideoWriter_fourcc(*'DIVX'), 20.0, frame_size)


    for f, image_file in enumerate(image_files):
        tic = cv2.getTickCount()

        if f == start_frame:  # init
            tracker.initialize(image_file, np.array(gt[f]))
            pred.append(gt[f])
            
            if save_video is not None:
                im = cv2.imread(image_file)
                cv2.rectangle(im, (int(gt[f][0]), int(gt[f][1])), (int(gt[f][0] + gt[f][2]), int(gt[f][1] + gt[f][3])), (0, 255, 0), 2)
                out.write(im)

        elif f > start_frame:  # tracking
            pred_bbox = tracker.track(image_file)
            b_overlap = scoring.get_precision(gt[f], pred_bbox)
            if b_overlap > 0:
                pred.append(pred_bbox)
            else:
                pred.append(2)
                start_frame = f + 5

            if verbose > 1:
                print("{} gt: {} pred: {} overlap: {}".format(f, gt[f], pred_bbox, b_overlap))

            if save_video is not None:
                im = cv2.imread(image_file)
                cv2.rectangle(im, (int(gt[f][0]), int(gt[f][1])), (int(gt[f][0] + gt[f][2]), int(gt[f][1] + gt[f][3])), (0, 255, 0), 2)
                cv2.rectangle(im, (int(pred_bbox[0]), int(pred_bbox[1])), (int(pred_bbox[0] + pred_bbox[2]), int(pred_bbox[1] + pred_bbox[3])), (0, 0, 255), 2)
                out.write(im)

        toc += cv2.getTickCount() - tic

    toc /= cv2.getTickFrequency()

    precisions = [scoring.get_precision(gt[i], pred[i]) for i in range(len(gt))]
    precisions = np.array(precisions)
    mprec = np.mean(precisions)

    success = [scoring.is_success(gt[i], pred[i]) for i in range(len(gt))]
    success = np.array(success)
    msucc = np.mean(success)

    fps = f / toc

    if verbose > 0:
        print('Video: {:12s} Time: {:2.1f}s Speed: {:5.2f}fps mSuccess: {:3.2f} mPrecision {:3.2f}'.format(video['name'], toc, f / toc, msucc, mprec))
    
    if save_video is not None:
        out.release()

    return mprec, msucc, fps

In [25]:
track_video(t1, v1, verbose=1, save_video='SEsiamFC_mytc128_Cup.avi')

Video: Cup          Time: 40.9s Speed:  7.39fps mSuccess: 1.00 mPrecision 0.81


(0.8127891293539966, 1.0, 7.3858280901800315)

In [11]:
def track_dataset(tracker, dataset, dataset_name, n = 1, verbose=0, save_results=False):
    precisions = []
    success = []
    fpss = []
    k = 0

    if save_results:
        if not os.path.exists(os.path.join("results")):
            os.makedirs(os.path.join("results"))

        if not os.path.exists(os.path.join("results", tracker.model_name)):
            os.makedirs(os.path.join("results", tracker.model_name))

    j = {}

    for video in dataset:
        if k == n:
            break

        precision, succes, fps = track_video(tracker, dataset[video], verbose)

        precisions.append(precision)
        success.append(succes)
        fpss.append(fps)

        if save_results:
            j[video] = {'precision': precision, 'succes': succes, 'fps': fps}

        k += 1

    if save_results:
        with open(os.path.join("results", tracker.model_name, dataset_name + '.json'), 'w') as f:
            json.dump(j, f, indent=2)

    return np.mean(precisions), np.mean(success), np.mean(fpss)

In [23]:
track_dataset(t1, d1, 'mytc128', verbose=1, save_results=True)
track_dataset(t1, d1, 'mytc128', verbose=1, save_results=True)

Video: Airport_ce   Time: 20.6s Speed:  7.14fps mSuccess: 0.91 mPrecision 0.78


(0.7812066443927109, 0.9054054054054054, 7.135532508335685)

In [1]:
def do_benchmark(models, datasets, verbose=0, save_results=False):
    for model_name in models:
        for dataset_name in datasets:
            print("Benchmarking {} on {}".format(model_name, dataset_name))
            track_dataset(models[model_name], datasets[dataset_name], dataset_name, verbose, save_results)